In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [0]:
x=pd.read_csv('https://s3.amazonaws.com/drivendata/data/44/public/dengue_features_train.csv')
y=pd.read_csv('https://s3.amazonaws.com/drivendata/data/44/public/dengue_labels_train.csv')
x_test=pd.read_csv('https://s3.amazonaws.com/drivendata/data/44/public/dengue_features_test.csv')
submission_file=pd.read_csv('https://s3.amazonaws.com/drivendata/data/44/public/submission_format.csv')

In [0]:
x.fillna(method='ffill', inplace=True)
x_test.fillna(method='ffill', inplace=True)

In [0]:
features=['ndvi_ne','year','weekofyear','ndvi_se','ndvi_nw','reanalysis_air_temp_k','precipitation_amt_mm']
fe=['year','weekofyear','station_avg_temp_c','reanalysis_dew_point_temp_k','ndvi_ne','reanalysis_tdtr_k']

In [0]:
X=x[fe]
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1456 entries, 0 to 1455
Data columns (total 6 columns):
year                           1456 non-null int64
weekofyear                     1456 non-null int64
station_avg_temp_c             1456 non-null float64
reanalysis_dew_point_temp_k    1456 non-null float64
ndvi_ne                        1456 non-null float64
reanalysis_tdtr_k              1456 non-null float64
dtypes: float64(4), int64(2)
memory usage: 68.3 KB


In [0]:
y.drop(columns='city',inplace=True)
y.drop(columns='year',inplace=True)
y.drop(columns='weekofyear',inplace=True)

In [0]:
from sklearn.model_selection import train_test_split
seed = 0
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

In [0]:
from xgboost import XGBRegressor
my_model = XGBRegressor(learning_rate=0.009,n_estimators=1500,subsample=1,max_depth=7,cosample_bytree=1,gamma=1)
my_model.fit(X_train, y_train)

[21:15:20] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, cosample_bytree=1, gamma=1,
             importance_type='gain', learning_rate=0.009, max_delta_step=0,
             max_depth=7, min_child_weight=1, missing=None, n_estimators=1500,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [0]:
from sklearn.metrics import mean_absolute_error
predictions = my_model.predict(X_test)

print("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_test)))

Mean Absolute Error: 9.662770768142726


In [0]:
model_full=XGBRegressor(learning_rate=0.009,n_estimators=1500,subsample=1,max_depth=7,cosample_bytree=1,gamma=1)
model_full.fit(X,y)
x_test=x_test[fe]
x.head()
x_test.head()

[21:15:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


,year,weekofyear,station_avg_temp_c,reanalysis_dew_point_temp_k,ndvi_ne,reanalysis_tdtr_k
0,2008,18,26.528571,294.527143,-0.0189,3.128571
1,2008,19,26.071429,294.395714,-0.0180,2.571429
2,2008,20,27.928571,295.308571,-0.0015,4.428571
3,2008,21,28.057143,294.402857,-0.0015,4.342857
4,2008,22,27.614286,294.760000,0.0568,3.542857


In [0]:
#x_test['city1']=np.where(x_test['city']=='sj',1,-1)
#x_test.drop(columns='city',inplace=True)

In [0]:
test_pred=model_full.predict(x_test).astype(int)
submission=pd.read_csv('https://s3.amazonaws.com/drivendata/data/44/public/submission_format.csv',index_col=[0,1,2])
submission.total_cases=test_pred
submission.to_csv('value.csv')

In [0]:
from sklearn.ensemble import RandomForestRegressor
rf=RandomForestRegressor()
rf.fit(X_train,y_train)


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [0]:
from sklearn.metrics import mean_absolute_error
predictions = rf.predict(X_test)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_test)))

Mean Absolute Error: 10.837328767123287
